In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import sklearn
import shutil

In [ ]:
!pip install urlextract --quiet
from urlextract import URLExtract
import requests
import torchvision.models as models
from torchvision import transforms

In [ ]:
# !pip install Katna -q
# from Katna.video import Video
# from Katna.writer import KeyFrameDiskWriter

In [ ]:
!pip install paddlepaddle-gpu -q
!pip install paddleocr -q
!pip install timeout_decorator

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from timeout_decorator import timeout
import shutil

In [ ]:
def get_content_type(url):
    try:
        response = requests.head(url)
        content_type = response.headers.get('Content-Type')
        return content_type
    except requests.ConnectionError:
        return None

def is_image_url(url):
    content_type = get_content_type(url)
    return content_type and content_type.startswith('image')

def is_video_url(url):
    content_type = get_content_type(url)
    return content_type and content_type.startswith('video')

In [ ]:
def download_image(url, destination_path):
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Open a temporary file to save the video content
            with open('temp_image.jpg', 'wb') as temp_file:
                # Copy the content from the response to the temporary file
                shutil.copyfileobj(response.raw, temp_file)

            # Move the temporary file to the destination path
            shutil.move('temp_image.jpg', destination_path)
            print(f"Image downloaded successfully to '{destination_path}'")
        else:
            print(f"Failed to download Image. Status code: {response.status_code}")
    except requests.ConnectionError:
        print("Connection error. Failed to download image.")

In [ ]:
def download_video(url, destination_path):
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Open a temporary file to save the video content
            with open('temp_video.mp4', 'wb') as temp_file:
                # Copy the content from the response to the temporary file
                shutil.copyfileobj(response.raw, temp_file)

            # Move the temporary file to the destination path
            shutil.move('temp_video.mp4', destination_path)
        else:
            print(f"Failed to download video. Status code: {response.status_code}")
    except requests.ConnectionError:
        print("Connection error. Failed to download video.")

In [ ]:
ocr_model = PaddleOCR(lang='en', use_angle_cls=True)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:13<00:00, 293kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:16<00:00, 603kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:11<00:00, 186kiB/s]

[2023/12/08 10:17:55] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/

In [ ]:
# def video_text(url):
#   download_video(url, "/content/temp_video.mp4")
#   video_path = "/content/temp_video.mp4"
#   cap = cv2.VideoCapture(video_path)
#   total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#   num_frames = 50
#   total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#   frame_interval = max(1, total_frames // num_frames)
#   video_ocr = []

#   for frame_count in range(0,total_frames,frame_interval):
#     ret, frame = cap.read()
#     if not ret:
#       break

#     result = ocr_model.ocr(frame)
#     extracted_text = []

#     for sublist in result:
#       if sublist is None:
#         break
#       for subsublist in sublist:
#         text = subsublist[1][0]
#         extracted_text.append(text)
#     text = "".join(extracted_text)

#     video_ocr.append(text)

#   cap.release()
#   return ' ' + (' ').join(video_ocr) + ' '

In [ ]:
def image_text(url):
  download_image(url, "/content/temp_image.jpg")
  image_path = "/content/temp_image.jpg"
  image = cv2.imread(image_path)
  result = ocr_model.ocr(image)
  extracted_text = []

  for sublist in result:
    if sublist is None:
      break
    for subsublist in sublist:
      text = subsublist[1][0]
      extracted_text.append(text)
  text_output = "".join(extracted_text)

  return text_output

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Adobe_PS/all_captions_upto_50k.csv")

In [ ]:
train_df["image_ocr"] = [""]*len(train_df)

In [ ]:
count_check = 0
for i in range(len(train_df)):
  print("*************************",i,"*****************************")
  url = train_df["single_media"][i]
  if is_image_url(url):
    count_check += 1
    text = image_text(url)
    print(text)
    train_df["image_ocr"][i] = text
  if count_check % 50 == 0:
    train_df.to_csv("all_captions_upto_50k.csv")